# 海洋データを用いて、気象予測を行う

## 背景
> 海洋データを用いて、scikit-learnのシュミレーションをやりたい  
> 海洋データはRINEXと呼ばれるデータ形式でした  
>まず、こちらのデータを読み出すサンプルを試す  
>  
> GPSファイル読み出しサンプル  
> 森下功啓製作所 様のサンプルを参考にさせて頂きました  
>  http://morimori2008.web.fc2.com/contents/GPS/GPS_programing/gps_tools.html
>

## 状況
* うまく動作しない。。なぜだろう。
* python2.7だと駄目かな。。
* それほど深追いしていないけど。。


## GPS座標データを読み込み表示する

In [17]:
# -*- coding:utf-8 -*-
#-------------------------------------------------------------------------------
# Purpose: 測位衛星の位置座標を求める。
#-------------------------------------------------------------------------------

import os,sys
#ライブラリパスカスタム指定
sys.path.append('../../notebook/pylib_20140228_morishita/')

import gnss.ephemeris as gnss_eph          # エフェメリス全般を担当するモジュール
import gnss.gps.ephemeris as gps_eph    # GPSのエフェメリスを担当するモジュール
import gnss.gps.time as gtime                   # GPSにおける時刻を担当するモジュール
import timeKM                                              #　一般的な時刻関係の処理を担当するモジュール
import datetime                                           # Pythonに標準で含まれる時刻関連モジュール

def create_filter(epoch):
    """ '渡されたエフェメリスを用いて、有効なエフェメリスを用いて各衛星座標を返す'関数を生成して返す
    応用を考慮しているので、GPSだけの計算に比べて少し行数が多くなっています。
    Argv:
        epoch: <float or int> GPS時刻
    """
    def filter(system_name, ephs_dict):
        """ 渡されたエフェメリスを用いて、有効なエフェメリスを用いて各衛星座標を返す
        Argv:
            system_name: <str> 測位システム名, e.g. "GPS", 本関数では利用しない
            ephs_dict: <dict>  エフェメリスを衛星ごとに格納した辞書, e.g.: {"1": [ephemeris・・・], ・・・}
        Return:
            <list<tuple<int, xyz>>> 衛星番号と座標をタプルにしたリスト
        """
        poss = []                                                       # 結果を格納するリスト
        for sat_name in ephs_dict:
            eph_list = ephs_dict[sat_name]                              # エフェメリスのリストを取得
            for eph in eph_list:
                if system_name == "GPS":                                # GPSの場合
                    if eph.sv_health != 0:
                        continue
                    tom = eph.TOM + eph.gps_week * timeKM.TIME_A_WEEK   # TOMをGPS時刻へ変換
                    if not (tom < epoch < tom + 3500 * eph.fit_interval):
                        continue
                if system_name == "QZS":                                # QZSの場合
                    #if eph.sv_health != 0:                             # sv_health is always 1 at 2014-02
                    #    continue
                    tom = eph.TOM + eph.gps_week * timeKM.TIME_A_WEEK   # TOMをGPS時刻へ変換
                    if not (tom < epoch < tom + 3500 * eph.fit_interval):
                        continue
                pos = eph.calc_sat_position(epoch)                      # 衛星の位置座標を計算
                #print(pos)
                poss.append((sat_name, pos))
                break
        return poss
    return filter

def main(src,tgt):
    srcpath=src;
    trtpath=tgt;
    
    eph_reader = gps_eph.reader()                                                              # エフェメリスを読み込むオブジェクトを生成
    eph_list = eph_reader.read_ephemeris_from_dir(srcpath)               # フォルダを指定して、エフェメリスをリストとして取得
    print eph_list
    
    if len(eph_list) > 0:                                                                                    # 読み込みに成功していることを確認
        with open(trtpath, "w", encoding="utf-8-sig") as fw:                      # ファイルに書き出す準備
            fw.write("GPS time,PRN,x,y,z\n")
            sgmr = gnss_eph.sub_manager("GPS")              # エフェメリスを管理するオブジェクトを生成
            sgmr.add_ephemeris(eph_list)                            # 管理オブジェクトにエフェメリスのリストを渡す

            start = datetime.datetime(2013, 7, 11, 0, 0, 0) # time system is UTC.
            end   = datetime.datetime(2013, 7, 19, 0, 0, 0)
            epoch = start
            while epoch < end:                              #　指定された時間が来るまでループ
                gpstime = gtime.convert_utc2gpst(epoch)     # GPSTとUTCとの間に存在する15 ns程度のバイアスは無視
                print(epoch)
                func = create_filter(gpstime)
                poss = sgmr.get_anything(func)              # 衛星位置を計算（計算んを行わせる関数オブジェクトを渡している）
                for pos in poss:                            # 計算できた衛星の座標を全て表示しつつファイルへ保存
                    name, position = pos
                    txt = "{0},{1}".format(name, position)
                    print(txt)
                    fw.write(str(epoch) + "," + txt + "\n") # ファイルへ書き出し
                epoch += datetime.timedelta(seconds=900)    # 時刻を進める

if __name__ == '__main__':
    src='/Users/pshanmori/notebook/satellite_position/ephemerides'
    tgt='../../notebook/satellite_position/sat position.csv'
    main(src,tgt)


[]
